# Simultaneous Regression Run

Imports

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.linear_model import SGDRegressor
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

Load and Describe Data

In [ ]:
def load_pp_data():
    csv_path = r"C:\Users\18123\OneDrive\Documents\IU Bloomington\Machine-Learning-Project\PowerPlantData\CCPP\Folds5x2_pp.csv"
    return pd.read_csv(csv_path)

pp = load_pp_data()
print(pp.describe())

Split Train/Test Data

In [ ]:
pp["AT_cat"] = pd.cut(pp["AT"],bins=[0.,10.,20.,30.,np.inf],labels=[1,2,3,4])

split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index, test_index in split.split(pp,pp["AT_cat"]):
    train_set = pp.loc[train_index]
    test_set = pp.loc[test_index]

for set_ in(train_set,test_set):
    set_.drop("AT_cat",axis=1,inplace=True)

pptrain = train_set.copy()
pptest = test_set.copy()

pptrain_attrib = pptrain.drop("PE",axis=1)
pptrain_labels = pptrain["PE"].copy()
pptest_attrib = pptest.drop("PE",axis=1)
pptest_labels = pptest["PE"].copy()

scaler = StandardScaler()
scaler.fit_transform(pptrain_attrib)

Dictionary of Regression Hyperparamter Ranges

In [ ]:
regression_param_dict = {
DecisionTreeRegressor: [{'ccp_alpha': sp_randFloat(0,1), 'criterion': ['squared_error','friedman_mse','absolute_error','poisson'], 'max_depth': sp_randInt(1,100), 'max_features': [None,'sqrt','log2'], 'max_leaf_nodes': sp_randInt(2,100), 'min_impurity_decrease': sp_randFloat(0,1), 'min_samples_leaf': sp_randInt(1,100), 'min_samples_split': sp_randInt(2,100), 'min_weight_fraction_leaf': sp_randFloat(0,0.5), 'random_state': [10], 'splitter': ['best','random']}],
LinearRegression: [{'copy_X': [True], 'fit_intercept': [True, False], 'n_jobs': [None], 'normalize': ['deprecated'], 'positive': [True,False]}],
GaussianProcessRegressor: [{'alpha': sp_randFloat(1e-11,1e-9), 'copy_X_train': [True], 'kernel': [None], 'n_restarts_optimizer': sp_randInt(0,10), 'normalize_y': [True,False], 'optimizer': ['fmin_l_bfgs_b'], 'random_state': [10]}],
neighbors.KNeighborsRegressor: [{'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': sp_randInt(1,100), 'metric': ['minkowski'], 'metric_params': [None], 'n_jobs': [None], 'n_neighbors': sp_randInt(1,50), 'p': sp_randInt(1,2), 'weights': ['uniform','distance']}],
RandomForestRegressor: [{'bootstrap': [True,False], 'ccp_alpha': sp_randFloat(0,1), 'criterion': ['squared_error', 'absolute_error', 'poisson'], 'max_depth': sp_randInt(1,100), 'max_features': [None,'sqrt','log2'], 'max_leaf_nodes': sp_randInt(2,100), 'max_samples': [None], 'min_impurity_decrease': sp_randFloat(0,1), 'min_samples_leaf': sp_randInt(1,100), 'min_samples_split': sp_randInt(2,100), 'min_weight_fraction_leaf': sp_randFloat(0,0.5), 'n_estimators': sp_randInt(1,50), 'n_jobs': [None], 'oob_score': [False], 'random_state': [10], 'verbose': [0], 'warm_start': [False]}]
}

Function that Runs Regressions Simultaneously

In [ ]:
def run(model):
    reg = model()
    param_grid = regression_param_dict[model]
    reg_hyper_search = RandomizedSearchCV(reg,param_grid,scoring="neg_mean_squared_error",cv=10,n_iter=2,refit=True,random_state=12)
    final_reg = reg_hyper_search.fit(pptrain_attrib,pptrain_labels)

    final_reg_train_predictions = final_reg.predict(pptrain_attrib)
    final_reg_train_mse = mean_squared_error(pptrain_labels,final_reg_train_predictions)
    final_reg_train_rmse = np.sqrt(final_reg_train_mse)
    final_reg_train_r2 = r2_score(pptrain_labels,final_reg_train_predictions)
    final_reg_train_mae = mean_absolute_error(pptrain_labels,final_reg_train_predictions)

    final_reg_test_predictions = final_reg.predict(pptest_attrib)

    final_reg_test_mse = mean_squared_error(pptest_labels,final_reg_test_predictions)
    final_reg_test_rmse = np.sqrt(final_reg_test_mse)
    final_reg_test_r2 = r2_score(pptest_labels,final_reg_test_predictions)
    final_reg_test_mae = mean_absolute_error(pptest_labels,final_reg_test_predictions)

    return [final_reg.best_params_, final_reg_train_mse, final_reg_train_rmse, final_reg_train_r2, final_reg_train_mae, final_reg_test_mse, final_reg_test_rmse, final_reg_test_r2, final_reg_test_mae]


def vizualizeMSE(regressions,modellst,data):
    MSEfig = plt.figure()
    for i in range(len(regressions)):
        if data == 'train':
            plt.scatter(regressions[i][1],i+1,s=100)
        if data == 'test':
            plt.scatter(regressions[i][5],i+1,s=100)
    plt.xticks([*range(0,31,2)])
    plt.yticks([*range(len(modellst)+2)],labels=['',*modellst,''])
    plt.xlabel(f'{data} Mean Squared Error (Lower is better)')
    return MSEfig

def vizualizeRMSE(regressions,modellst,data):
    RMSEfig = plt.figure()
    for i in range(len(regressions)):
        if data == 'train':
            plt.scatter(regressions[i][2],i+1,s=100)
        if data == 'test':
            plt.scatter(regressions[i][6],i+1,s=100)
    xticklst = []
    for j in range(0,75,5):
        xticklst += [j/10]
    plt.xticks(xticklst)
    plt.yticks([*range(len(modellst)+2)],labels=['',*modellst,''])
    plt.xlabel(f'{data} Root Mean Squared Error (Lower is better)')
    return RMSEfig

def vizualizeR2(regressions,modellst,data):
    R2fig = plt.figure()
    for i in range(len(regressions)):
        if data == 'train':
            plt.scatter(regressions[i][3],i+1,s=100)
        if data == 'test':
            plt.scatter(regressions[i][7],i+1,s=100)
    xticklst = []
    for j in range(70,102,2):
        xticklst += [j/100]
    plt.xticks(xticklst)
    plt.yticks([*range(len(modellst)+2)],labels=['',*modellst,''])
    plt.xlabel(f'{data} R-Squared Score (Higher is better)')
    return R2fig

def vizualizeMAE(regressions,modellst,data):
    MAEfig = plt.figure()
    for i in range(len(regressions)):
        if data == 'train':
            plt.scatter(regressions[i][4],i+1,s=100)
        if data == 'test':
            plt.scatter(regressions[i][8],i+1,s=100)
    xticklst = []
    for j in range(0,625,25):
        xticklst += [j/100]
    plt.xticks(xticklst)
    plt.yticks([*range(len(modellst)+2)],labels=['',*modellst,''])
    plt.xlabel(f'{data} Mean Absolute Error (Lower is better)')
    return MAEfig

def comparison(modellst):
    regressions = []
    for i in modellst:
        regressions += [run(i)]
    trainmse = vizualizeMSE(regressions,modellst,'train')
    trainrmse = vizualizeRMSE(regressions,modellst,'train')
    trainr2 = vizualizeR2(regressions,modellst,'train')
    trainmae = vizualizeMAE(regressions,modellst,'train')
    testmse = vizualizeMSE(regressions,modellst,'test')
    testrmse = vizualizeRMSE(regressions,modellst,'test')
    testr2 = vizualizeR2(regressions,modellst,'test')
    testmae = vizualizeMAE(regressions,modellst,'test')
    return trainmse, trainrmse, trainr2, trainmae, testmse, testrmse, testr2, testmae

x = [DecisionTreeRegressor,LinearRegression,GaussianProcessRegressor,RandomForestRegressor,neighbors.KNeighborsRegressor]
comparison(x)
plt.show()